In [141]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots
import math
from typing import Literal

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [142]:
untracked_list = [
    dt.date(2024, 5, 1),
    dt.date(2024, 4, 17),
    
]

async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1) and (f_today <= dt.date(2024, 3, 28)):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    else:
        days_to_wednesday = (2 - f_today.weekday()) % 7
        nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
        f_expiry = nearest_wednesday
        if nse.valid_days(
            start_date=nearest_wednesday, end_date=nearest_wednesday
        ).empty:
            f_expiry = nearest_wednesday - dt.timedelta(days=1)

    if f_expiry in untracked_list:
        return f_expiry - dt.timedelta(days=1)
    else:
        return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_sensex(f_today):

    days_to_thursday = (4 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [179]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
bnf_1min = pd.read_csv("../data/nifty.csv")
# bnf_1min = pd.read_csv('../data/nifty_min.csv')
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2017) & (bnf_1min["datetime"].dt.year <= 2024)
]

In [180]:
bnf_1min.head()

,datetime,open,high,low,close,volume
0,2017-01-02 09:15:00,8210.1000,8211.7000,8189.0000,8189.5500,0
1,2017-01-02 09:16:00,8188.7500,8193.9500,8188.7500,8189.9500,0
2,2017-01-02 09:17:00,8190.1500,8190.7500,8173.7000,8173.7000,0
3,2017-01-02 09:18:00,8173.3500,8177.5500,8169.1500,8177.5500,0
4,2017-01-02 09:19:00,8177.8500,8178.1500,8173.4500,8174.4000,0


In [181]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [182]:
data = resample(pl.DataFrame(bnf_1min), "1d")
data = data.to_pandas()
data['daily_range'] = data['high'] - data['low']
data.tail()

,datetime,open,high,low,close,daily_range
1837,2024-06-03,23337.9000,23338.7000,23062.3000,23305.9500,276.4000
1838,2024-06-04,23179.5000,23179.5000,21281.4500,21996.4500,1898.0500
1839,2024-06-05,22128.3500,22670.4000,21791.9500,22573.0000,878.4500
1840,2024-06-06,22798.6000,22910.1500,22642.6000,22849.0500,267.5500
1841,2024-06-07,22821.8500,23320.2000,22789.0500,23270.0500,531.1500


In [185]:
PORTFOLIO = 1_00_00_000

INDEX_ = 'nifty'
STRIKE_SPREAD_ = 50
LOT_SIZE_ = 25
LEVERAGE_ = 6
INSTRUMENT_NAME_ = 'NIFTY'

In [187]:
async def generate_tradebook(df, index, mult):

    results = []
    excluded_dates = [
        dt.date(2020, 6, 4),
        dt.date(2023, 2, 22),
        
    ]

    for i in range(1, len(df)):
        current_datetime = df.iloc[i]['datetime']
        
        print(current_datetime.date())

        if current_datetime.date() not in excluded_dates:
            # current_candle_open = df.iloc[i]['open']
            current_candle_high = df.iloc[i]['high']
            current_candle_low = df.iloc[i]['low']
            current_candle_close = df.iloc[i]['close']
            
            previous_day_high = df.iloc[i-1]['high']
            previous_day_low = df.iloc[i-1]['low']
            previous_day_range = df.iloc[i-1]['daily_range']
            multiplier = mult * previous_day_range
            
            df_today = bnf_1min[bnf_1min['datetime'].dt.date == current_datetime.date()]                # This will contain 1min data for only iterating date
            if len(df_today)>1:
                # print(df_today.head())
                current_candle_open = df_today.iloc[1]['open']                                              # This will give us 09:16 open
                
                high_level = current_candle_open + multiplier
                low_level = current_candle_open - multiplier
                
                breakout_high = df_today[df_today["high"] >= high_level]
                breakout_low = df_today[df_today["low"] <= low_level]
        
                if not breakout_high.empty:
                    breakout_high_time = breakout_high.iloc[0]["datetime"]
                    atm_strike = int(round(low_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)
        
                    expiry = await get_expiry_nifty(current_datetime.date())
                    dte = (expiry - breakout_high_time.date()).days
                    contract = await get_option_contract_name(
                        symbol=INSTRUMENT_NAME_,
                        strike=atm_strike,
                        expiry=expiry,
                        opt_type="PE",
                    )
                    pe_df = await fetch_data(
                        index=INDEX_,
                        start_date=current_datetime.date(),
                        end_date=current_datetime.date(),
                        start_time=breakout_high_time.time(),
                        end_time=dt.time(15, 30),
                        expiry=expiry,
                        strike=atm_strike,
                        asset_class="P",
                    )
                    if not isinstance(pe_df, str) and pe_df is not None :
                        pe_df = pe_df.to_pandas()
                        entry_price = pe_df.iloc[0]["c"]
                        # sl_price = entry_price * (1 + sl_pct)
                    else:
                        entry_price = float("nan")
        
                    # Exit Scenario
                    # data_after_entry = df_today[
                    #     (df_today["datetime"] > breakout_high_time)
                    #     & (
                    #         df_today["datetime"].dt.date
                    #         <= current_datetime.date()
                    #     )
                    # ]
                    if not isinstance(pe_df, str) and pe_df is not None :
                        option_data_after_entry = pe_df[
                            (pe_df["datetime"] > breakout_high_time)
                            & (
                                pe_df["datetime"].dt.date
                                <= current_datetime.date()
                            )
                        ]
    
                        data_after_entry = df_today[df_today['datetime'] > breakout_high_time]
        
                        max_high = option_data_after_entry['h'].max()
                        max_low = option_data_after_entry['l'].min()
        
                        # pe_sl_breach = option_data_after_entry[option_data_after_entry['h'] >= sl_price]
                        pe_sl_breach = data_after_entry[data_after_entry['low'] <= low_level]
                    
                    # print(current_week_data_after_entry)
                    # low_breach = data_after_entry[
                    #     data_after_entry["low"] <= previous_day_low
                    # ]
                    # # print(low_breach)
                    # if len(low_breach) > 0:
                    #     exit_time = low_breach.iloc[0]["datetime"]
                    #     # print(exit_time)
                    #     remark = "SL Hit"
                    #     # print(remark, exit_time)
                    #     # print(pe_df)
                    #     if not math.isnan(entry_price):
                    #         # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    #         exit_price = pe_df.loc[
                    #             (pe_df["datetime"].dt.date >= exit_time.date())
                    #             & (pe_df["datetime"].dt.hour >= exit_time.hour)
                    #             & (pe_df["datetime"].dt.minute >= exit_time.minute),
                    #             "c",
                    #         ].iloc[0]
        
                    #     else:
                    #         exit_price = float("nan")
                    #         # print(exit_price)
                        if len(pe_sl_breach) > 0:
                            exit_time = pe_sl_breach.iloc[0]["datetime"]
                            remark = "SL Hit"
            
                            if not math.isnan(entry_price):
                                exit_price = pe_df.loc[pe_df['datetime'] >= exit_time, 'c'].iloc[0]
                            else:
                                exit_price = float("nan")
                        
                        else:
                            exit_time = dt.datetime.combine(
                                expiry - dt.timedelta(days=0), dt.time(15, 20)
                            )
                            remark = "EOD Closing"
                            # print(remark, exit_time)
                            if not math.isnan(entry_price):
                                exit_row = pe_df.loc[pe_df['datetime'] >= dt.datetime.combine(current_datetime.date(), dt.time(15,20)) , 'c']
                                if len(exit_row) > 0:
                                    exit_price = exit_row.iloc[0]
                                else:
                                    exit_price = float('nan')
                            else:
                                exit_price = float("nan")
                                # print(exit_price)
                    
                    # print(exit_time)
                    # print(exit_price)
                    # print(contract)
                    
                    if not pd.isna(entry_price):
                        qty = PORTFOLIO * LEVERAGE_ / atm_strike
                        slippage = 0.01 * (entry_price + exit_price)
                        pnl = qty * (entry_price - exit_price)
                        final_pnl = qty * (entry_price - exit_price - slippage)
                        unit = {
                            "Date": current_datetime.date(),
                            "Previous Day High": previous_day_high,
                            "Previous Day Low": previous_day_low,
                            "Daily Range": previous_day_range,
                            "Today Open": current_candle_open,
                            "Multiplier to Range": f"{int(mult * 100)}%",
                            "Entry Level Long": high_level,
                            "Entry Level Short": low_level,
                            "Break Type": "high",
                            "Strike": atm_strike,
                            "Expiry": expiry,
                            "DTE": dte,
                            "Contract": contract,
                            "Option Type": contract[-2:],
                            "Entry Time": breakout_high_time,
                            "Entry Price": entry_price,
                            # "Initial SL": sl_price,
                            "Exit Time": exit_time,
                            "Exit Price": exit_price,
                            "Max High": max_high,
                            "Max Low": max_low,
                            "Remark": remark,
                            "Points": entry_price - exit_price,
                            "Slippage": slippage,
                            "Points w cs": (entry_price - exit_price) - slippage,
                            'Leverage': LEVERAGE_,
                            "Qty": qty,
                            "PnL": pnl,
                            "PnL w cs": final_pnl,
                            "ROI%": (pnl * 100 / PORTFOLIO),
                            "ROI% w cs": (final_pnl * 100 / PORTFOLIO),
                        }
                        # print(unit)
                        results.append(unit)
        
                if not breakout_low.empty:
                    breakout_low_time = breakout_low.iloc[0]["datetime"]
                    atm_strike = int(round(high_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)
        
                    expiry = await get_expiry_nifty(current_datetime.date())
                    dte = (expiry - breakout_low_time.date()).days
                    contract = await get_option_contract_name(
                        symbol=INSTRUMENT_NAME_,
                        strike=atm_strike,
                        expiry=expiry,
                        opt_type="CE",
                    )
                    ce_df = await fetch_data(
                        index=INDEX_,
                        start_date=current_datetime.date(),
                        end_date=current_datetime.date(),
                        start_time=breakout_low_time.time(),
                        end_time=dt.time(15, 30),
                        expiry=expiry,
                        strike=atm_strike,
                        asset_class="C",
                    )
                    if not isinstance(ce_df, str) and ce_df is not None :
                        ce_df = ce_df.to_pandas()
                        entry_price = ce_df.iloc[0]["c"]
                        # sl_price = entry_price * (1 + sl_pct)
        
                    else:
                        entry_price = float("nan")
        
                    # Exit Scenario
                    # data_after_entry = df_today[
                    #     (df_today["datetime"] > breakout_low_time)
                    #     & (
                    #         df_today["datetime"].dt.date
                    #         <= current_datetime.date()
                    #     )
                    # ]
                    # # print(current_week_data_after_entry)
                    # high_breach = data_after_entry[
                    #     data_after_entry["high"] >= previous_day_high
                    # ]
                    # # print(low_breach)
                    # if len(high_breach) > 0:
                    #     exit_time = high_breach.iloc[0]["datetime"]
                    #     # print(exit_time)
                    #     remark = "SL Hit"
                    #     # print(remark, exit_time)
                    #     # print(ce_df)
                    #     if not math.isnan(entry_price):
                    #         # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    #         exit_price = ce_df.loc[
                    #             (ce_df["datetime"].dt.date >= exit_time.date())
                    #             & (ce_df["datetime"].dt.hour >= exit_time.hour)
                    #             & (ce_df["datetime"].dt.minute >= exit_time.minute),
                    #             "c",
                    #         ].iloc[0]
        
                    #     else:
                    #         exit_price = float("nan")
                    #         # print(exit_price)
        
                    if not isinstance(ce_df, str) and ce_df is not None :
                        option_data_after_entry = ce_df[
                            (ce_df["datetime"] > breakout_low_time)
                            & (
                                ce_df["datetime"].dt.date
                                <= current_datetime.date()
                            )
                        ]
    
                        data_after_entry = df_today[df_today['datetime'] > breakout_low_time]
        
                        max_high = option_data_after_entry['h'].max()
                        max_low = option_data_after_entry['l'].min()
            
                        # ce_sl_breach = option_data_after_entry[option_data_after_entry['h'] >= sl_price]
                        ce_sl_breach = data_after_entry[data_after_entry['high'] >= high_level]
                        
            
                        if len(ce_sl_breach) > 0:
                            exit_time = ce_sl_breach.iloc[0]["datetime"]
                            remark = "SL Hit"
            
                            if not math.isnan(entry_price):
                                exit_price = ce_df.loc[ce_df['datetime'] >= exit_time, 'c'].iloc[0]
                            else:
                                exit_price = float("nan")
            
                        else:
                            exit_time = dt.datetime.combine(
                                expiry - dt.timedelta(days=0), dt.time(15, 20)
                            )
                            remark = "EOD Closing"
                            # print(remark, exit_time)
                            if not math.isnan(entry_price):
                                exit_row = ce_df.loc[ce_df['datetime'] >= dt.datetime.combine(current_datetime.date(), dt.time(15,20)) , 'c']
                                if len(exit_row) > 0:
                                    exit_price = exit_row.iloc[0]
                                else:
                                    exit_price = float('nan')
                            else:
                                exit_price = float("nan")
                                # print(exit_price)
                    
                    # print(exit_time)
                    # print(exit_price)
                    # print(contract)
                    
                    # print(entry_price)
                    if not pd.isna(entry_price):
                        qty = PORTFOLIO * LEVERAGE_ / atm_strike
                        slippage = 0.01 * (entry_price + exit_price)
                        pnl = qty * (entry_price - exit_price)
                        final_pnl = qty * (entry_price - exit_price - slippage)
                        unit = {
                            "Date": current_datetime.date(),
                            "Previous Day High": previous_day_high,
                            "Previous Day Low": previous_day_low,
                            "Daily Range": previous_day_range,
                            "Today Open": current_candle_open,
                            "Multiplier to Range": f"{int(mult * 100)}%",
                            "Entry Level Long": high_level,
                            "Entry Level Short": low_level,
                            "Break Type": "low",
                            "Strike": atm_strike,
                            "Expiry": expiry,
                            "DTE": dte,
                            "Contract": contract,
                            "Option Type": contract[-2:],
                            "Entry Time": breakout_low_time,
                            "Entry Price": entry_price,
                            # "Initial SL": sl_price,
                            "Exit Time": exit_time,
                            "Exit Price": exit_price,
                            "Max High": max_high,
                            "Max Low": max_low,
                            "Remark": remark,
                            "Points": entry_price - exit_price,
                            "Slippage": slippage,
                            "Points w cs": (entry_price - exit_price) - slippage,
                            'Leverage': LEVERAGE_,
                            "Qty": qty,
                            "PnL": pnl,
                            "PnL w cs": final_pnl,
                            "ROI%": (pnl * 100 / PORTFOLIO),
                            "ROI% w cs": (final_pnl * 100 / PORTFOLIO),
                        }
                        # print(unit)
                        results.append(unit)
    
    return results

In [204]:
async def trade():
    trades = await generate_tradebook(data, INDEX_, 0.7)
    return pd.DataFrame(trades)


In [205]:
tb = await trade()

2017-01-03
               datetime      open      high       low     close  volume
375 2017-01-03 09:15:00 8195.2500 8196.0000 8172.2500 8173.4500       0
376 2017-01-03 09:16:00 8173.1500 8173.7500 8168.6500 8170.4500       0
377 2017-01-03 09:17:00 8170.2000 8170.2000 8163.1500 8167.2500       0
378 2017-01-03 09:18:00 8166.5500 8169.2000 8165.7500 8167.8500       0
379 2017-01-03 09:19:00 8168.0000 8168.1500 8163.8500 8164.3500       0
2017-01-04
               datetime      open      high       low     close  volume
750 2017-01-04 09:15:00 8200.8000 8201.6000 8194.6500 8201.4000       0
751 2017-01-04 09:16:00 8202.4000 8211.2000 8201.9000 8210.3500       0
752 2017-01-04 09:17:00 8210.1000 8210.7500 8209.2000 8209.8500       0
753 2017-01-04 09:18:00 8209.6500 8209.6500 8204.5000 8205.0500       0
754 2017-01-04 09:19:00 8204.9500 8209.2000 8204.9500 8207.7500       0
2017-01-05
                datetime      open      high       low     close  volume
1125 2017-01-05 09:15:00 8226.

In [206]:
tradebook = pd.DataFrame(tb)

In [207]:
tradebook['ROI% w cs'].sum()

46.115769133080185

In [208]:
# tradebook = tb
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [209]:
tradebook.head()

,Date,Previous Day High,Previous Day Low,Daily Range,Today Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Max High,Max Low,Remark,Points,Slippage,Points w cs,Leverage,Qty,PnL,PnL w cs,ROI%,ROI% w cs,Trade Year
0,2017-01-20,8445.0500,8404.3000,40.7500,8408.2500,70%,8436.7750,8379.7250,low,8450,2017-01-25,5,NIFTY17JAN8450CE,CE,2017-01-20 13:00:00,15.4000,2017-01-25 15:20:00,13.2000,18.1500,11.5500,EOD Closing,2.2000,0.2860,1.9140,6,7100.5917,15621.3018,13590.5325,0.1562,0.1359,2017
1,2017-01-23,8423.4500,8341.0000,82.4500,8342.5000,70%,8400.2150,8284.7850,high,8300,2017-01-25,2,NIFTY17JAN8300PE,PE,2017-01-23 11:05:00,7.4000,2017-01-25 15:20:00,7.4500,16.7000,6.4500,EOD Closing,-0.0500,0.1485,-0.1985,6,7228.9157,-361.4458,-1434.9398,-0.0036,-0.0143,2017
2,2017-01-24,8403.9500,8328.0500,75.9000,8415.4500,70%,8468.5800,8362.3200,high,8350,2017-01-25,1,NIFTY17JAN8350PE,PE,2017-01-24 14:31:00,2.2500,2017-01-25 15:20:00,2.0500,2.3500,1.7500,EOD Closing,0.2000,0.0430,0.1570,6,7185.6287,1437.1257,1128.1437,0.0144,0.0113,2017
3,2017-01-25,8480.7500,8399.4500,81.3000,8501.9000,70%,8558.8100,8444.9900,high,8450,2017-01-25,0,NIFTY17JAN8450PE,PE,2017-01-25 14:29:00,0.1500,2017-01-25 15:20:00,0.0500,0.1500,0.0500,EOD Closing,0.1000,0.0020,0.0980,6,7100.5917,710.0592,695.8580,0.0071,0.0070,2017
4,2017-02-17,8783.8000,8719.9500,63.8500,8866.0500,70%,8910.7450,8821.3550,low,8900,2017-02-23,6,NIFTY17FEB8900CE,CE,2017-02-17 09:41:00,20.2500,2017-02-23 15:20:00,19.3500,32.5000,16.2000,EOD Closing,0.9000,0.3960,0.5040,6,6741.5730,6067.4157,3397.7528,0.0607,0.0340,2017


In [210]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% w cs"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0]["ROI% w cs"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0]["ROI% w cs"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% w cs"].cumsum() - combined_df_sorted["ROI% w cs"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [211]:
stats = generate_stats(tradebook, f'Multiplier : {int(i*100)}%')
for overall_roi_dd_ratio, stats_df in stats.items():
    print(overall_roi_dd_ratio)
    print(stats_df.to_string())
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio < -10:
        stats_dict[overall_roi_dd_ratio] = stats_df
        print(stats_df.to_string())

4.2014716054829115
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio         Variation
2017       1.5305           38  60.5263                0.2074             -0.2160      -2.3111       0.6623  Multiplier : 35%
2018       1.8645           30  70.0000                0.2439             -0.3620      -2.8543       0.6532  Multiplier : 35%
2019       3.9183          116  69.8276                0.3345             -0.6623      -6.5463       0.5986  Multiplier : 35%
2020      20.3700          136  65.4412                0.5608             -0.6285      -5.1132       3.9838  Multiplier : 35%
2021       9.8939          139  70.5036                0.3232             -0.5312      -4.9300       2.0069  Multiplier : 35%
2022       8.1604          130  73.0769                0.3594             -0.7424      -5.1538       1.5834  Multiplier : 35%
2023      -4.4999          139  64.7482                0.1584             -0.3827      -6.7250     

In [ ]:
# tradebook.to_csv('Daily RBW var2.csv', index=False)

In [79]:
stats_dict = {}

multipliers = [0.35]
for i in multipliers:
    print(f'Multiplier : {int(i*100)}%')
    tb = await generate_tradebook(data, INDEX_, i)
    tradebook = pd.DataFrame(tb)
    tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
    tradebook["Trade Year"] = tradebook["Entry Time"].dt.year
    stats = generate_stats(tradebook, f'Multiplier : {int(i*100)}%')
    for overall_roi_dd_ratio, stats_df in stats.items():
        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 10:
            stats_dict[overall_roi_dd_ratio] = stats_df
            # Print the stats
            print('Overall Stats Combined')
            print(stats_df.to_string())

Multiplier : 35%
